In [1]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('./important_csvs/')
sys.path.append('../video-classification/ResNetCRNN/')

from helpers_lstm import *
from helpers_training import *

In [2]:
tensor_transform = get_tensor_transform('ImageNet', True)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
df = pd.read_csv('./small_dataset_csvs/events_with_number_of_frames_stratified.csv')
df_train = get_df(df, 20, True, False, False)
class_image_paths, end_idx = get_indices(df_train, root_dir)
train_loader = get_loader(16, 128, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, True, False)
df_valid = get_df(df, 20, False, True, False)
class_image_paths, end_idx = get_indices(df_valid, root_dir)
valid_loader = get_loader(16, 128, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, True, False)
df_test = get_df(df, 20, False, False, True)
class_image_paths, end_idx = get_indices(df_test, root_dir)
test_loader = get_loader(16, 128, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, True, False)


In [3]:
device = torch.device('cuda')
cnn_encoder = ResCNNEncoder().to(device)
adaptive_pool = AdaptiveConcatPool2d()
cnn_encoder.resnet[8] = adaptive_pool
for param in cnn_encoder.parameters():
    param.requires_grad = False
for param in cnn_encoder.resnet[8].parameters():
    param.requires_grad = True
for param in cnn_encoder.headbn1.parameters():
    param.requires_grad = True
for param in cnn_encoder.fc1.parameters():
    param.requires_grad = True
    
rnn_decoder = DecoderRNNattention(batch_size=128).to(device)
for param in rnn_decoder.parameters():
    param.requires_grad = True

In [4]:
crnn_params, cnn_encoder, rnn_decoder = parallelize_model(cnn_encoder, rnn_decoder)

model = nn.Sequential(cnn_encoder,rnn_decoder)
torch.cuda.empty_cache()

Using 4 GPUs!


In [5]:
checkpoint = torch.load('/media/scratch/astamoulakatos/saved-lstm-models/first-round-same-dataset/best-checkpoint-000epoch.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print('loading pretrained freezed model!')

loading pretrained freezed model!


In [6]:
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X, y in valid_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        y = y.squeeze(dim=1)
        y = y.float()
        output = model(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        preds = preds.to(torch.float32) 
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)

In [7]:
y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [8]:
y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [9]:
from helpers_thresholds import *

In [10]:
classes = ['Exposure', 'Burial', 'Field Joint', 'Anode', 'Free Span']

In [11]:
thresholds = nsea_compute_thresholds(y_tr, y_pr, classes)

In [12]:
thresholds

{'Exposure': 0.41521564,
 'Burial': 0.5350713,
 'Field Joint': 0.48112452,
 'Anode': 0.47195727,
 'Free Span': 0.54344684}

## validation set

In [13]:
new_compute_metrics(y_tr, y_pr, thresholds, classes)

,Event,Threshold,Exact Matching Score,Precision,Recall,F1-Score,tn,fp,fn,tp
0,Exposure,0.415216,0.893945,0.922861,0.924677,0.923768,1287.0,275.0,268.0,3290.0
1,Burial,0.535071,0.901172,0.771885,0.959667,0.855594,3115.0,443.0,63.0,1499.0
2,Field Joint,0.481125,0.916602,0.686464,0.713056,0.699507,4196.0,227.0,200.0,497.0
3,Anode,0.471957,0.843555,0.233645,0.781250,0.359712,4094.0,738.0,63.0,225.0
4,Free Span,0.543447,0.973828,0.969549,0.958087,0.963784,3203.0,56.0,78.0,1783.0
5,Aggregate,NaN,0.646875,0.807484,0.915641,0.858168,NaN,NaN,NaN,NaN


# test set

In [14]:
torch.cuda.empty_cache()

In [15]:
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        y = y.squeeze(dim=1)
        y = y.float()
        output = model(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        preds = preds.to(torch.float32) 
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)

In [16]:
y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [17]:
y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [18]:
new_compute_metrics(y_tr, y_pr, thresholds, classes)

,Event,Threshold,Exact Matching Score,Precision,Recall,F1-Score,tn,fp,fn,tp
0,Exposure,0.415216,0.722842,0.724013,0.920901,0.810673,696.0,1216.0,274.0,3190.0
1,Burial,0.535071,0.787202,0.690099,0.729079,0.709054,2838.0,626.0,518.0,1394.0
2,Field Joint,0.481125,0.865699,0.450450,0.722311,0.554871,4204.0,549.0,173.0,450.0
3,Anode,0.471957,0.725074,0.151707,0.638298,0.245148,3658.0,1342.0,136.0,240.0
4,Free Span,0.543447,0.974516,0.927415,0.995803,0.960393,3578.0,130.0,7.0,1661.0
5,Aggregate,NaN,0.462054,0.642249,0.862240,0.736161,NaN,NaN,NaN,NaN
